<a href="https://colab.research.google.com/github/hakmin1015/PowerSystemEconomics/blob/main/etc/AI_LSTM/LSTM_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM Develop 3

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/LoadForecasting/

Mounted at /content/drive/
/content/drive/MyDrive/LoadForecasting


## 2023년 전체 테스트

2일 전까지 학습하여 예측하는 모델 <br/>
하루의 체감온도 및 합계일사량으로 max값을 예측함. <br/>
max 대신 min을 사용하면 최소값을 예측할 수 있음. <br/>
모델 학습 시간에 굉장히 오랜 시간이 소요됨.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Multiply

# 엑셀 파일에서 데이터를 읽어옴
df = pd.read_excel('jeju_asos_data_4.xlsx')

# 시간에 따라 데이터를 정렬
df.sort_values(by='date', inplace=True)

start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 12, 31)

# 각 날짜에 대해 반복 작업 수행
for input_date in pd.date_range(start=start_date, end=end_date):

    # train과 validation set을 2021년 1월 1일부터 입력된 날짜 기준 이틀 전까지로 설정
    train_validation_set = df[(df['date'] >= '2021-01-01') & (df['date'] < input_date - datetime.timedelta(days=2))]

    # test set은 input_date 하루만
    test_set = df[df['date'] == input_date]

    # 결과 확인
    # print("Train-validation set 크기:", len(train_validation_set))
    # print("Test set 크기:", len(test_set))

    # 전체 데이터에서 'p-temp', 'solar'와 'max' 컬럼만 사용
    data = train_validation_set[['p-temp', 'solar', 'max']].values

    # Min-Max 스케일링
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)

    # 시퀀스 데이터 생성 함수
    def create_sequences(data, seq_length):
        X, y = [], []
        for i in range(len(data) - seq_length):
            X.append(data[i:i+seq_length, :-1])
            y.append(data[i+seq_length, -1])
        return np.array(X), np.array(y)

    # 하이퍼파라미터 설정
    sequence_length = 30  # 시퀀스 길이

    # 시퀀스 데이터 생성
    X, y = create_sequences(data_scaled, sequence_length)

    # 가중치 정의
    weights = np.array([0.8, 0.2])  # 가중치 적용 : p-temp / solar

    # 입력 데이터에 가중치 적용
    X_weighted = X * weights

    # LSTM 모델 정의
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_weighted.shape[1], X_weighted.shape[2])),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])

    # 모델 컴파일
    model.compile(optimizer='adam', loss='mse')

    # 모델 학습
    history = model.fit(X_weighted, y, epochs=50, batch_size=32, verbose=0, validation_split=0.1)

    # Loss 그래프 출력
    # plt.plot(history.history['loss'], label='Training Loss')
    # plt.plot(history.history['val_loss'], label='Validation Loss')
    # plt.xlabel('Epoch')
    # plt.ylabel('Loss')
    # plt.title('Training and Validation Loss')
    # plt.legend()
    # plt.show()

    # 예측 함수
    def predict_power_for_date(date, model, scaler, sequence_length, weights):
        # 입력 날짜에 해당하는 인덱스 찾기
        idx = df[df['date'] == date].index[0] - 2

        # 이전 sequence_length일의 p-temp와 solar 값 추출
        X_input = data_scaled[idx-sequence_length:idx, :-1]

        # 입력 데이터에 가중치 적용
        X_input_weighted = X_input * weights

        # 입력 데이터를 LSTM 모델에 맞게 변환
        X_input_weighted = X_input_weighted.reshape((1, sequence_length, X_input_weighted.shape[1]))

        # 예측
        prediction = model.predict(X_input_weighted)

        # 스케일링된 예측값을 디스케일링
        last_sequence = np.concatenate((X_input_weighted[0, -1, :], prediction[0])).reshape(1, -1)
        predicted_power = scaler.inverse_transform(last_sequence)[0, -1]

        return predicted_power

    print("예측하는 날짜:", input_date.strftime('%Y-%m-%d'))  # 해당하는 날짜 출력

    # 예측
    predicted_power = predict_power_for_date(input_date, model, scaler, sequence_length, weights)
    print("예측한 최고 전력 : ", predicted_power)

    # 실제 최고 기온 출력
    actual_power = test_set['max'].values[0]
    print("실제 최고 전력 : ", actual_power)

    # 오차율 출력
    error_rate = (predicted_power - actual_power) / actual_power * 100
    print("오차율(%) : ", error_rate)
    print('\n')

예측하는 날짜: 2023-01-01
1/1 [==============================] - 5s 5s/step
예측한 최고 전력 :  1007.1878863573073
실제 최고 전력 :  859
오차율(%) :  17.25120912192169


예측하는 날짜: 2023-01-02
1/1 [==============================] - 1s 751ms/step
예측한 최고 전력 :  956.6926088929175
실제 최고 전력 :  989
오차율(%) :  -3.266672508299543


예측하는 날짜: 2023-01-03
1/1 [==============================] - 1s 797ms/step
예측한 최고 전력 :  982.4994649887084
실제 최고 전력 :  976
오차율(%) :  0.6659287898266786


예측하는 날짜: 2023-01-04
1/1 [==============================] - 1s 777ms/step
예측한 최고 전력 :  1007.5215522050856
실제 최고 전력 :  963
오차율(%) :  4.6232141438302845


예측하는 날짜: 2023-01-05


1/1 [==============================] - 1s 825ms/step
예측한 최고 전력 :  1003.1762645840644
실제 최고 전력 :  943
오차율(%) :  6.381364218882754


예측하는 날짜: 2023-01-06


1/1 [==============================] - 1s 1s/step
예측한 최고 전력 :  969.8106302618979
실제 최고 전력 :  912
오차율(%) :  6.33888489713793


예측하는 날짜: 2023-01-07
1/1 [==============================] - 1s 760ms/step
예측한 최고 전력 :  952.9220953583716
실제 최고 전력 :  935
오차율(%) :  1.9168016426065904


예측하는 날짜: 2023-01-08
1/1 [==============================] - 1s 781ms/step
예측한 최고 전력 :  989.5401330590247
실제 최고 전력 :  891
오차율(%) :  11.059498659823198


예측하는 날짜: 2023-01-09
1/1 [==============================] - 1s 779ms/step
예측한 최고 전력 :  947.0242104530333
실제 최고 전력 :  908
오차율(%) :  4.297820534475036


예측하는 날짜: 2023-01-10
1/1 [==============================] - 1s 830ms/step
예측한 최고 전력 :  936.3399705886841
실제 최고 전력 :  902
오차율(%) :  3.80709208300267


예측하는 날짜: 2023-01-11
1/1 [==============================] - 1s 790ms/step
예측한 최고 전력 :  933.4064730405807
실제 최고 전력 :  909
오차율(%) :  2.6849805325171343


예측하는 날짜: 2023-01-12
1/1 [==============================] - 1s 826ms/step
예측한 최고 전력 :  933.0000414252281
실제 최고 전력 :  846
오차

KeyboardInterrupt: 